<a href="https://colab.research.google.com/github/John-Curcio/finetuning/blob/main/cumtown_subreddit.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!apt-get install zstd

Reading package lists... Done
Building dependency tree       
Reading state information... Done
The following NEW packages will be installed:
  zstd
0 upgraded, 1 newly installed, 0 to remove and 15 not upgraded.
Need to get 343 kB of archives.
After this operation, 1,592 kB of additional disk space will be used.
Get:1 http://archive.ubuntu.com/ubuntu focal-updates/universe amd64 zstd amd64 1.4.4+dfsg-3ubuntu0.1 [343 kB]
Fetched 343 kB in 2s (200 kB/s)
Selecting previously unselected package zstd.
(Reading database ... 123069 files and directories currently installed.)
Preparing to unpack .../zstd_1.4.4+dfsg-3ubuntu0.1_amd64.deb ...
Unpacking zstd (1.4.4+dfsg-3ubuntu0.1) ...
Setting up zstd (1.4.4+dfsg-3ubuntu0.1) ...
Processing triggers for man-db (2.9.1-1) ...


In [10]:
!zstd -d Cumtown_comments.zst
# !zstd -d Cumtown_submissions.zst

Cumtown_comments.zst: 661931047 bytes 


In [11]:
!ls

Cumtown_comments  Cumtown_comments.zst	sample_data


In [12]:
!head -n 10 Cumtown_comments

{"ups":2,"stickied":false,"id":"d6mt4xx","author_flair_css_class":null,"score":2,"gilded":0,"subreddit_id":"t5_3f2nu","subreddit":"Cumtown","edited":false,"retrieved_on":1473475519,"link_id":"t3_4y812t","author_flair_text":null,"controversiality":0,"created_utc":1471539246,"body":"Holy shit this was terrible. 45 minutes of talking about some rape case nobody has ever heard of. Do comedy next week, love ya guys.","distinguished":null,"author":"MosTheBoss","parent_id":"t3_4y812t"}
{"author":"XmoonmanX","parent_id":"t3_4x4fgl","ups":1,"author_flair_css_class":null,"stickied":false,"subreddit":"Cumtown","controversiality":0,"author_flair_text":null,"subreddit_id":"t5_3f2nu","created_utc":1472768097,"gilded":0,"retrieved_on":1475246682,"edited":false,"distinguished":null,"body":"I thought Stavros was a last name","score":1,"link_id":"t3_4x4fgl","id":"d75zvwd"}
{"author":"OedipusOrca","parent_id":"t3_50s31n","ups":1,"author_flair_css_class":null,"subreddit":"Cumtown","stickied":false,"contro

In [5]:
# !git clone https://huggingface.co/datasets/eli5

In [6]:
!pip install -q -U bitsandbytes
!pip install -q -U git+https://github.com/huggingface/transformers.git
!pip install -q -U git+https://github.com/huggingface/peft.git
!pip install -q -U git+https://github.com/huggingface/accelerate.git
!pip install -q datasets
!pip install -q sentencepiece

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 97.1/97.1 MB 16.0 MB/s eta 0:00:00
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 236.8/236.8 kB 14.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.8/7.8 MB 107.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 71.3 MB/s eta 0:00:00
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 227.6/227.6 kB 15.7 MB/s eta 0:00:00
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 486.2/486.2 kB 24.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 110.5/110.5 kB 14.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━

In [33]:
import torch
from transformers import AutoTokenizer, AutoModelForCausalLM, BitsAndBytesConfig

# model_id = "EleutherAI/gpt-neox-20b"
# model_id = "decapoda-research/llama-7b-hf" # start with a small model
model_id = "openlm-research/open_llama_7b"
bnb_config = BitsAndBytesConfig(
    load_in_4bit=True,
    bnb_4bit_use_double_quant=True,
    bnb_4bit_quant_type="nf4",
    bnb_4bit_compute_dtype=torch.bfloat16
)

tokenizer = AutoTokenizer.from_pretrained(model_id, tokenizer_class="LlamaTokenizer",)
# tokenizer = AutoTokenizer.from_pretrained(model_id)
model = AutoModelForCausalLM.from_pretrained(model_id, quantization_config=bnb_config, device_map={"":0})

Loading checkpoint shards:   0%|          | 0/46 [00:00<?, ?it/s]

In [13]:
import pandas as pd
import json
from datasets import Dataset

def load_comments_dataset(path="Cumtown_comments"):
    data = open(path, "r").read()
    comments = data.split("\n")
    comments_json = []
    for comment in comments[:-1]:
        try:
            comments_json.append(json.loads(comment))
        except:
            print(f"Error parsing comment: {comment}")
    comments_df = pd.DataFrame(comments_json)
    comments_df["created_ts"] = pd.to_datetime(comments_df["created_utc"], unit="s")
    comments_df["edited"] = comments_df["edited"].astype(bool)
    return Dataset.from_pandas(comments_df)

def load_submissions_dataset(path="Cumtown_submissions"):
    data = open(path, "r").read()
    submissions = data.split("\n")
    submissions_json = []
    for submission in submissions[:-1]:
        try:
            submissions_json.append(json.loads(submission))
        except:
            print(f"Error parsing submission: {submission}")
    submissions_df = pd.DataFrame(submissions_json)
    submissions_df["created_ts"] = pd.to_datetime(submissions_df["created_utc"], unit="s")
    submissions_df["edited"] = submissions_df["edited"].astype(bool)
    return Dataset.from_pandas(submissions_df)

comments_df = load_comments_dataset("Cumtown_comments")
# submissions_df = load_submissions_dataset("Cumtown_submissions")
# submissions_df

In [14]:
# for row in submissions_df:
#     if row["score"] > 200 and row["is_self"]:
#         print(row)
#         break

In [15]:
# pd.Series(submissions_df["is_self"]).sum()

In [16]:
# import sys
# del sys.modules["transformers"]
# # del moduleName

# Let's try QLoRA

In [34]:
from peft import prepare_model_for_kbit_training

model.gradient_checkpointing_enable()
model = prepare_model_for_kbit_training(model)

In [35]:
def print_trainable_parameters(model):
    """
    Prints the number of trainable parameters in the model.
    """
    trainable_params = 0
    all_param = 0
    for _, param in model.named_parameters():
        all_param += param.numel()
        if param.requires_grad:
            trainable_params += param.numel()
    print(
        f"trainable params: {trainable_params} || all params: {all_param} || trainable%: {100 * trainable_params / all_param}"
    )

In [36]:
model.__dict__.keys()

dict_keys(['training', '_parameters', '_buffers', '_non_persistent_buffers_set', '_backward_pre_hooks', '_backward_hooks', '_is_full_backward_hook', '_forward_hooks', '_forward_hooks_with_kwargs', '_forward_pre_hooks', '_forward_pre_hooks_with_kwargs', '_state_dict_hooks', '_state_dict_pre_hooks', '_load_state_dict_pre_hooks', '_load_state_dict_post_hooks', '_modules', 'config', 'name_or_path', 'warnings_issued', 'generation_config', '_is_quantized_training_enabled', 'is_8bit_serializable', '_is_hf_initialized', 'is_loaded_in_4bit', 'is_loaded_in_8bit', 'is_quantized', '_old_forward', '_hf_hook', 'forward', 'hf_device_map', '_require_grads_hook'])

In [37]:
model._modules

OrderedDict([('gpt_neox',
              GPTNeoXModel(
                (embed_in): Embedding(50432, 6144)
                (layers): ModuleList(
                  (0-43): 44 x GPTNeoXLayer(
                    (input_layernorm): LayerNorm((6144,), eps=1e-05, elementwise_affine=True)
                    (post_attention_layernorm): LayerNorm((6144,), eps=1e-05, elementwise_affine=True)
                    (attention): GPTNeoXAttention(
                      (rotary_emb): RotaryEmbedding()
                      (query_key_value): Linear4bit(in_features=6144, out_features=18432, bias=True)
                      (dense): Linear4bit(in_features=6144, out_features=6144, bias=True)
                    )
                    (mlp): GPTNeoXMLP(
                      (dense_h_to_4h): Linear4bit(in_features=6144, out_features=24576, bias=True)
                      (dense_4h_to_h): Linear4bit(in_features=24576, out_features=6144, bias=True)
                      (act): FastGELUActivation()
          

In [38]:
print_trainable_parameters(model)

trainable params: 0 || all params: 10588901376 || trainable%: 0.0


In [39]:
from peft import LoraConfig, get_peft_model

config = LoraConfig(
    r=8,
    lora_alpha=32,
    target_modules=["query_key_value", "embed_out"],
    # target_modules=["q_proj", "k_proj", "v_proj"],
    lora_dropout=0.05,
    bias="none",
    task_type="CAUSAL_LM"
)

model = get_peft_model(model, config)
print_trainable_parameters(model)

trainable params: 9103360 || all params: 10598004736 || trainable%: 0.0858969233055455


In [23]:
tokenizer("Hello my name is")["input_ids"]

[12092, 619, 1416, 310]

In [24]:
# from datasets import load_dataset


# data = load_dataset("Abirate/english_quotes")
data = comments_df.filter(
    lambda example: example["score"] > 50 and \
    len(tokenizer.__call__(example["body"])) < 2048 # max length allowed
 ).map(
    lambda samples: tokenizer.__call__(samples["body"]),
    batched=True
)

Filter:   0%|          | 0/698508 [00:00<?, ? examples/s]

Map:   0%|          | 0/24150 [00:00<?, ? examples/s]

In [25]:
len(data)

24150

In [40]:
# text = "Hello my name is"
text = "Adam Friedland, Nick Mullen, Stavros"
device = "cuda:0"

inputs = tokenizer(text, return_tensors="pt").to(device)
outputs = model.generate(**inputs, max_new_tokens=10)
print(tokenizer.decode(outputs[0], skip_special_tokens=True))

Setting `pad_token_id` to `eos_token_id`:0 for open-end generation.


Adam Friedland, Nick Mullen, Stavros Stavrinides, and
    Vin


In [41]:
import transformers

# needed for gpt-neo-x tokenizer
tokenizer.pad_token = tokenizer.eos_token

trainer = transformers.Trainer(
    model=model,
    train_dataset=data.shuffle(seed=42),
    args=transformers.TrainingArguments(
        per_device_train_batch_size=1,
        # per_device_train_batch_size=4, # I think the huge batch size -> indexing, overflow errors
        gradient_accumulation_steps=4,
        warmup_steps=2,
        # max_steps=10**3,
        max_steps=100,
        # num_train_epochs=2,
        # max_steps=100,
        # max_steps=20,
        learning_rate=2e-4,
        # learning_rate=2e-5,
        # fp16=True,
        logging_steps=1,
        output_dir="outputs",
        optim="paged_adamw_8bit",
    ),
    data_collator=transformers.DataCollatorForLanguageModeling(tokenizer, mlm=False),
)
model.config.use_cache = False  # silence the warnings. Please re-enable for inference!
trainer.train()

You're using a GPTNeoXTokenizerFast tokenizer. Please note that with a fast tokenizer, using the `__call__` method is faster than using a method to encode the text followed by a call to the `pad` method to get a padded encoding.


Step,Training Loss
1,4.687600
2,5.604700
3,4.659700
4,4.544100
5,4.676800
6,4.499500
7,4.411200
8,3.865700
9,5.098500
10,4.883700


KeyboardInterrupt: ignored

In [42]:
model.config.use_cache = True  # silence the warnings. Please re-enable for inference!

# text = "Hello my name is"
text = "Adam Friedland, Nick Mullen, Stavros"
device = "cuda:0"

inputs = tokenizer(text, return_tensors="pt").to(device)
outputs = model.generate(**inputs, max_new_tokens=10)
print(tokenizer.decode(outputs[0], skip_special_tokens=True))

Setting `pad_token_id` to `eos_token_id`:0 for open-end generation.
`use_cache=True` is incompatible with gradient checkpointing. Setting `use_cache=False`...
/usr/local/lib/python3.10/dist-packages/torch/utils/checkpoint.py:31: UserWarning: None of the inputs have requires_grad=True. Gradients will be None
  warnings.warn("None of the inputs have requires_grad=True. Gradients will be None")
`use_cache=True` is incompatible with gradient checkpointing. Setting `use_cache=False`...
`use_cache=True` is incompatible with gradient checkpointing. Setting `use_cache=False`...
`use_cache=True` is incompatible with gradient checkpointing. Setting `use_cache=False`...
`use_cache=True` is incompatible with gradient checkpointing. Setting `use_cache=False`...
`use_cache=True` is incompatible with gradient checkpointing. Setting `use_cache=False`...
`use_cache=True` is incompatible with gradient checkpointing. Setting `use_cache=False`...
`use_cache=True` is incompatible with gradient checkpointin

Adam Friedland, Nick Mullen, Stavros retarded Lenin retarded retarded retarded retarded retarded retarded retarded retarded


In [43]:
# text = "Hello my name is"
text = "And I find it kind of funny, I find it kind of sad,"
device = "cuda:0"

inputs = tokenizer(text, return_tensors="pt").to(device)
outputs = model.generate(**inputs, max_new_tokens=20)
print(tokenizer.decode(outputs[0], skip_special_tokens=True))

Setting `pad_token_id` to `eos_token_id`:0 for open-end generation.
`use_cache=True` is incompatible with gradient checkpointing. Setting `use_cache=False`...
`use_cache=True` is incompatible with gradient checkpointing. Setting `use_cache=False`...
`use_cache=True` is incompatible with gradient checkpointing. Setting `use_cache=False`...
`use_cache=True` is incompatible with gradient checkpointing. Setting `use_cache=False`...
`use_cache=True` is incompatible with gradient checkpointing. Setting `use_cache=False`...
`use_cache=True` is incompatible with gradient checkpointing. Setting `use_cache=False`...
`use_cache=True` is incompatible with gradient checkpointing. Setting `use_cache=False`...
`use_cache=True` is incompatible with gradient checkpointing. Setting `use_cache=False`...
`use_cache=True` is incompatible with gradient checkpointing. Setting `use_cache=False`...
`use_cache=True` is incompatible with gradient checkpointing. Setting `use_cache=False`...
`use_cache=True` is in

And I find it kind of funny, I find it kind of sad, retarded retarded retarded closer retarded retarded retarded retarded retarded retarded retarded retarded retarded retarded retarded retarded retarded retarded retarded retarded


In [ ]:
# text = "Hello my name is"
text = "Evolution is"
device = "cuda:0"

inputs = tokenizer(text, return_tensors="pt").to(device)
outputs = model.generate(**inputs, max_new_tokens=200)
print(tokenizer.decode(outputs[0], skip_special_tokens=True))

In [ ]:
# text = "Hello my name is"
text = "Norm Macdonald is certified not a "
device = "cuda:0"

inputs = tokenizer(text, return_tensors="pt").to(device)
outputs = model.generate(**inputs, max_new_tokens=10)
print(tokenizer.decode(outputs[0], skip_special_tokens=True))

In [ ]:
# text = "Hello my name is"
text = "Norm Macdonald is certified NOT A"
device = "cuda:0"

inputs = tokenizer(text, return_tensors="pt").to(device)
outputs = model.generate(**inputs, max_new_tokens=10)
print(tokenizer.decode(outputs[0], skip_special_tokens=True))

In [ ]:
# text = "Hello my name is"
text = "NOT A"
device = "cuda:0"

inputs = tokenizer(text, return_tensors="pt").to(device)
outputs = model.generate(**inputs, max_new_tokens=10)
print(tokenizer.decode(outputs[0], skip_special_tokens=True))